# Set up environment
## Install libs

In [ ]:
!pip install -q plotly==5.20.0 python-dotenv openai==1.42.0 "httpx<0.28"


try:
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

## Clone git repository

In [ ]:
import os


try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB and ".git" not in os.listdir():
    !git init .
    !git remote add origin https://github.com/batprem/llm-balance-sheets
    !git pull origin main

# Install Thai font

In [ ]:
# Reference: https://medium.com/@arttantra.art/%E0%B9%80%E0%B8%9E%E0%B8%B4%E0%B9%88%E0%B8%A1%E0%B8%9F%E0%B8%AD%E0%B8%99%E0%B8%95%E0%B9%8C%E0%B9%84%E0%B8%97%E0%B8%A2%E0%B9%83%E0%B8%99-jupyter-notebook-add-thai-font-to-jupyter-notebook-916b4d95586b

import matplotlib
import matplotlib.font_manager as fm
from os import path
if not path.exists('Sarabun-Regular.ttf'):
    !wget https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf -O Sarabun-Regular.ttf

if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

# Set up environment variables

In [ ]:
if IN_COLAB:
    from google.colab import userdata

    GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
    TYPHOON_API_KEY = userdata.get("TYPHOON_API_KEY")
else:
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
    TYPHOON_API_KEY = os.environ.get("TYPHOON_API_KEY")

# Set up Typhoon API

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
from typing_extensions import TypedDict
from IPython.display import Markdown


MODEL = "typhoon-v1.5x-70b-instruct"
genai.configure(api_key=GEMINI_API_KEY)



client = OpenAI(
   api_key=TYPHOON_API_KEY,
   base_url='https://api.opentyphoon.ai/v1'
)


class ChatTurn(TypedDict):
    role: str
    content: str

## Set up Typhoon completion

In [ ]:
def get_open_ai_completion(
    prompt: str,
    model: str = MODEL,
    stream: bool = False,
    initial_message: list[ChatTurn]| None = None,
    temperature: float = 0
) -> str:
    if initial_message is None:
        initial_message = []
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model, messages=messages, temperature=temperature, stream=stream
    )
    if not stream:
        return response.choices[0].message.content
    else:
        result = ""
        for chunk in response:
            content = chunk.choices[0].delta.content
            print(content, end="")
            if isinstance(content, str):
                result += content
        return result

# Set up GEMINI completion

In [ ]:
def get_gemini_completion(
    prompt: str,
    stream: bool = False,
) -> str:
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text



# Run Balance sheet analytics
## Basic analyse

In [ ]:
prompt = """
The stock KBANK is a bussiness mentioned in Thai "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"


Balance sheet detail


| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากในประเทศ (พันบาท)       | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากต่างประเทศ (พันบาท)      | 0.00           | 0.00           | 0.00           |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้จากในประเทศ (%)           | 100.00         | 100.00         | 100.00         |
| รายได้จากต่างประเทศ (%)          | 0.00           | 0.00           | 0.00           |


Act as finanical analyst and analyse balance sheet in Markdown format
"""

In [ ]:
Markdown(get_open_ai_completion(prompt, stream=False))

In [ ]:
Markdown(get_gemini_completion(prompt, stream=False))

In [ ]:
prompt = """
The stock KBANK is a bussiness mentioned in Thai "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"


Balance sheet detail

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากในประเทศ (พันบาท)       | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากต่างประเทศ (พันบาท)      | 0.00           | 0.00           | 0.00           |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้จากในประเทศ (%)           | 100.00         | 100.00         | 100.00         |
| รายได้จากต่างประเทศ (%)          | 0.00           | 0.00           | 0.00           |

Analyse this data and write a Python code with Plotly visualise data.

You must give only the answer do not add prologue, prefix and suffix
"""

python_code = get_open_ai_completion(prompt, stream=True)
_ = exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
python_code = get_gemini_completion(prompt)
exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
prompt = """
The stock KBANK is a bussiness mentioned in Thai "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"


Balance sheet detail


| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากในประเทศ (พันบาท)       | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากต่างประเทศ (พันบาท)      | 0.00           | 0.00           | 0.00           |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้จากในประเทศ (%)           | 100.00         | 100.00         | 100.00         |
| รายได้จากต่างประเทศ (%)          | 0.00           | 0.00           | 0.00           |


Act as finanical analyst and analyse this data and write a Python code with Plotly to create at least 3 visualisations to ease investors.

You must give only the answer do not add prologue, prefix and suffix.
"""

python_code = get_open_ai_completion(prompt, stream=True)
exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
python_code = get_gemini_completion(prompt)
exec(python_code.strip("`").lstrip("python").strip())

## Chain of Thought

In [ ]:
prompt = """
The stock KBANK is a bussiness mentioned in Thai "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"


Balance sheet detail


| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากในประเทศ (พันบาท)       | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากต่างประเทศ (พันบาท)      | 0.00           | 0.00           | 0.00           |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้จากในประเทศ (%)           | 100.00         | 100.00         | 100.00         |
| รายได้จากต่างประเทศ (%)          | 0.00           | 0.00           | 0.00           |


Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Comment file header and show your thought how this data can support the investors
2. Use Plotly to create at least 3 visualisations to ease investors.
3. Use print function to conclude your data analytics

You must give only the answer do not add prologue, prefix and suffix.
"""

python_code = get_open_ai_completion(prompt, stream=True)
exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
python_code = get_gemini_completion(prompt, stream=True)
exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
prompt = """
The stock KBANK is a bussiness mentioned in Thai "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"


Balance sheet detail

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากในประเทศ (พันบาท)       | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากต่างประเทศ (พันบาท)      | 0.00           | 0.00           | 0.00           |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้จากในประเทศ (%)           | 100.00         | 100.00         | 100.00         |
| รายได้จากต่างประเทศ (%)          | 0.00           | 0.00           | 0.00           |


Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Comment file header and show your thought how this data can support the investors
2. Use Matplotlib to create at least 3 visualisations to ease investors. You should use subplot for presentation and rotate X-axis label with 45 degree.
3. Use from IPython.display import HTML, display functions to conclude your data analytics

You must give only the answer do not add prologue, prefix and suffix.
"""

python_code = get_open_ai_completion(prompt, stream=True)
exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
python_code = get_gemini_completion(prompt, stream=True)
exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
prompt = """
The stock KBANK is a bussiness mentioned in Thai "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"


Balance sheet detail:

โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ:

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้ดอกเบี้ยสุทธิ (%)             | 73.09          | 76.76          | 77.05          |
| รายได้ที่มิใช่ดอกเบี้ย (%)            | 26.91          | 23.24          | 22.95          |

โครงสร้างรายได้จากในประเทศและจากต่างประเทศ:

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้ดอกเบี้ยสุทธิ (%)             | 73.09          | 76.76          | 77.05          |
| รายได้ที่มิใช่ดอกเบี้ย (%)            | 26.91          | 23.24          | 22.95          |

รายได้อื่น ตามที่ระบุในงบการเงิน:
| detail                                      | 2564   | 2565   | 2566   |
|:--------------------------------------------|:-------|:-------|:-------|
| รายได้อื่นรวม (พันบาท)                          | 0.00   | 0.00   | 0.00   |
| รายได้อื่นจากการดำเนินงาน (พันบาท)               | 0.00   | 0.00   | 0.00   |
| รายได้อื่นนอกเหนือจากการดำเนินงาน (พันบาท)        | 0.00   | 0.00   | 0.00   |
| ค่าใช้จ่ายการทำวิจัยและพัฒนา (R&D) ในระยะเวลา 3 ปี | N/A    | N/A    | N/A    |
| ที่ผ่านมา                                      |        |        |        |


Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Comment file header and show your thought how this data can support the investors
2. Use Matplotlib to create at least 3 visualisations to ease investors. You should use subplot for presentation and rotate X-axis label with 45 degree.
3. Use from IPython.display import HTML, display functions to conclude your data analytics

You must give only the answer do not add prologue, prefix and suffix.
"""

python_code = get_open_ai_completion(prompt, stream=True)
exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
python_code = get_gemini_completion(prompt, stream=True)
exec(python_code.strip("`").lstrip("python").strip())

In [ ]:
prompt = """
The stock KBANK is a bussiness mentioned in Thai "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"


Balance sheet detail:

โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ:

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้ดอกเบี้ยสุทธิ (%)             | 73.09          | 76.76          | 77.05          |
| รายได้ที่มิใช่ดอกเบี้ย (%)            | 26.91          | 23.24          | 22.95          |

โครงสร้างรายได้จากในประเทศและจากต่างประเทศ:

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้ดอกเบี้ยสุทธิ (%)             | 73.09          | 76.76          | 77.05          |
| รายได้ที่มิใช่ดอกเบี้ย (%)            | 26.91          | 23.24          | 22.95          |

รายได้อื่น ตามที่ระบุในงบการเงิน:
| detail                                      | 2564   | 2565   | 2566   |
|:--------------------------------------------|:-------|:-------|:-------|
| รายได้อื่นรวม (พันบาท)                          | 0.00   | 0.00   | 0.00   |
| รายได้อื่นจากการดำเนินงาน (พันบาท)               | 0.00   | 0.00   | 0.00   |
| รายได้อื่นนอกเหนือจากการดำเนินงาน (พันบาท)        | 0.00   | 0.00   | 0.00   |
| ค่าใช้จ่ายการทำวิจัยและพัฒนา (R&D) ในระยะเวลา 3 ปี | N/A    | N/A    | N/A    |
| ที่ผ่านมา                                      |        |        |        |


What if Bank of Thailand reduce interate rate, how does it affect the company, answer in bullet points?
"""

answer = get_open_ai_completion(prompt, stream=True)
Markdown(answer)

In [ ]:
answer = get_gemini_completion(prompt, stream=True)
Markdown(answer)

In [ ]:
prompt = """
As as financial analyst, show me example of questions in bullet points to evaluate the balance sheet the company:

let's see the available data is limited and you can only:
- โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ:
- โครงสร้างรายได้จากในประเทศและจากต่างประเทศ:
- รายได้อื่น ตามที่ระบุในงบการเงิน:


KBANK: "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"
"""
question = get_gemini_completion(prompt, stream=True)
Markdown(question)

In [ ]:
prompt = f"""
The stock KBANK is a bussiness mentioned in Thai "ธนาคารกสิกรไทย จํากัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ หลักทรัพย์และธุรกิจท่ีเก่ียวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา จํานวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สําหรับทําธุรกรรมด้วยตนเอง 10,903 เครื่อง ครอบคลุมทุกพื้นท่ีและเพียงพอต่อความต้องการของลูกค้า นอกจากน้ี ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8 ประเทศ มีศูนย์กลางการดําเนินงานและให้บริการท่ีสํานักงานใหญ่"


Balance sheet detail:

โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ:

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้ดอกเบี้ยสุทธิ (%)             | 73.09          | 76.76          | 77.05          |
| รายได้ที่มิใช่ดอกเบี้ย (%)            | 26.91          | 23.24          | 22.95          |

โครงสร้างรายได้จากในประเทศและจากต่างประเทศ:

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้ดอกเบี้ยสุทธิ (%)             | 73.09          | 76.76          | 77.05          |
| รายได้ที่มิใช่ดอกเบี้ย (%)            | 26.91          | 23.24          | 22.95          |

รายได้อื่น ตามที่ระบุในงบการเงิน:
| detail                                      | 2564   | 2565   | 2566   |
|:--------------------------------------------|:-------|:-------|:-------|
| รายได้อื่นรวม (พันบาท)                          | 0.00   | 0.00   | 0.00   |
| รายได้อื่นจากการดำเนินงาน (พันบาท)               | 0.00   | 0.00   | 0.00   |
| รายได้อื่นนอกเหนือจากการดำเนินงาน (พันบาท)        | 0.00   | 0.00   | 0.00   |
| ค่าใช้จ่ายการทำวิจัยและพัฒนา (R&D) ในระยะเวลา 3 ปี | N/A    | N/A    | N/A    |
| ที่ผ่านมา                                      |        |        |        |


Act as the best financial analyst and try your best answer the following question for the information and you are allowed to infer the answer from your experiece
{question}
"""

answer = get_open_ai_completion(prompt, stream=True)
Markdown(answer)